In [1]:
import pandas as pd
from pymongo import MongoClient
import pickle
import re

def load_and_encoding(file):
    with open(file, "rb") as fp:
        a = pickle.load(fp)
        # transform list of dict to Dataframe with code and label
        b = pd.DataFrame(a)
        return b


Pre-Process

In [66]:
#-----Load Train Data-------------------------------------------------------------
with open("ChinaTimes201610_201712_politics_BayesTrain", "rb") as fp:
    a1 = pickle.load(fp)
    politics = pd.DataFrame.from_dict(a1)

with open("textOthers_BayesTrain", "rb") as fp:
    a2 = pickle.load(fp)
    others = pd.DataFrame.from_dict(a2)
    others["category"] = "others"

frames = [politics,others]
trainframe = pd.concat(frames)
#-----Load Test Data---------------------------------------------------------------- 
with open("LTN_Bayes_Test", "rb") as fp:
    a1 = pickle.load(fp)
    politics = pd.DataFrame.from_dict(a1)
    politics["category"] = "politics"

with open("textOthers_BayesTest", "rb") as fp:
    a2 = pickle.load(fp)
    others = pd.DataFrame.from_dict(a2)
    others["category"] = "others"

frames = [politics,others]
testframe = pd.concat(frames)

# -------Load Data Need to be Classified------------------------------------------------------------------------
with open("ChinaTimesRawData", "rb") as fp:
    a3 = pickle.load(fp)
    PredictTmp = pd.DataFrame.from_dict(a3)
    

# -------Making Duration Prediction Validation Data--------------------------------------------
client = MongoClient("mongodb://192.168.87.135:27017")
db = client["news"]
collection = db["validation"]   
news_list_temp = collection.find({})
validTmp = pd.DataFrame(list(news_list_temp))



# -------Politian Words from LDA -------------------------------------------
m_words ="軍方,飛彈,海軍,國軍,國防,軍事,軍人,陸軍,退役,潛艦,軍官,武器,憲兵,演習,將領,軍艦,飛機,國艦國造,慶富,潛艇,旗艦,獵雷艦案,美國,國際,總統,外交,國家,美方,中國,外交,歐巴馬,川普,巴拿馬,我國,安全,協定,南海,官員,協議,會談,亞太".split(",")
p_words = "馬英九,總統,王金平,總統府,副總統,前總統,府方,立法院長,施明德,副院長,勞工,工會,勞動部,勞基法,工資,薪資,勞動,勞資,一例一休,資方,團體,權益,雇主,工商,周休二日,修法,配套,陳菊,高雄市,民眾,黃國昌,遊行,掃街,投票,號召,時代力量,連署,呼籲,大遊行,凱道,高市府,機關,行政,審查,規範,立委,民進黨,國民黨,立法院,立院,黨團,國會,協商,條例,法案,修法,朝野,共識,綠委,立法,草案,監督,質詢,決議,委員,行政院,藍委,蘇嘉全,證所稅,江宜樺,內閣制,江揆,證交稅,修憲,育嬰假,總統制,二讀,陳冠宇,年金改革,軍公教,公務員,年金,公務人員,榮民,亞投行,政黨票,退撫,退輔會,孫立群,考試院,銓敘部,公教,公職,職等,青年,年輕,時代,陸客,賴清德,市長,邱太三,組閣,鄭運鵬,吳思瑤,爆炸案,民政局,郭國文,陳美伶,三法,高分院,執委,孤島,南海,華航,太平島,海域,仲裁,我國,林全,行政院,內閣,閣揆,主委,出任,部會,交接,政院,請辭,行政院長,接任,外界,政策,上任,祕書長,首長,發言人,,抗議,社運,太陽花學運,抗爭,訴求,街頭,公民,群眾,公宅,聯盟,聲援,服貿,謝文定,高喊,占領,自救會,遴選,監委,林聰賢,縣府,蕭美琴,公平會,自民黨,田智宣,徐榛蔚,司法,判決,法務部,特偵組,大法官,司法院,審理,地院,最高法院,司改,人權,違憲,合議庭,司改會,蔡英文,台電,能源,台化,電價,用愛發電,發電,台塑,經濟部,供電,綠能,環保,減碳,核四,再生能源,太陽能,非核家園,核一,林佳龍,市府,花博,農損,南院,洪慈庸,治理,綠營,執政,補選,賄選,李進勇,藍綠,小英,兆豐銀,徐光曦,原民會,國銀,南沙群島,遠雄,大巨蛋,檢調,地檢署,洗錢,公安,北檢,選民,選戰,民調,黨中央,競選,選情,輔選,政黨,黨員,初選,黨產,不當黨產,政治獻金,夏立言,柯文哲,台北市,北市府,浩鼎,柯市府,雙城論壇,沙海林,郝龍斌,世大運,朱立倫,八仙,李全教,雙北,侯友宜,楊志良,李四川,呂忠吉,國內產業,降價,通路,聯合會,官股,王廷升,戴季全,同業公會".split(",")
PH_words = "醫療,醫院,醫師,醫療,衛福部,治療,病患,醫學,疾病,患者,病人,健保,手術,健康,醫護,保外就醫,就醫,醫生,汙染,汙染,環保署,環保,環境,空汙,空氣品質,食安,食品,消費者,頂新,食安,衛生局,檢驗,認證,稽查,檢測,食藥署,衛福部,國際衛生,感染,美豬,衛生,病毒,防疫,瘦肉精,病例,衛福部,防治,疫苗,健康,流行,生態,溼地,黑面琵鷺,黑琵".split(",")
cat_words = "風災,颱風,救災,重建,強風,風災,災情慘重,空難,飛機,逃生,客機,機師".split(",")
# c = re.match(".*民進黨.*",b.iloc[1,2])
# b.iloc[1,2]

In [67]:
def column_append(data,catName,cat):
    wordCount = 0
    catCount = 0
    for j in cat:
        for i in j:
            wordCount +=1
            colname = catName[catCount]+str(wordCount)
            col = data["content"].str.match(".*"+i+".*")
            data[colname]=col
        wordCount = 0
        catCount += 1
    return data


In [68]:
name = ['m_words','p_words','PH_words','cat_words']
cat = [m_words,p_words,PH_words,cat_words]
traindata = column_append(trainframe,name,cat)
testdata = column_append(testframe,name,cat)
PredictData = column_append(PredictTmp,name,cat)
ValidData = column_append(validTmp,name,cat)


Try Algorithm

In [70]:
from sklearn.naive_bayes import BernoulliNB
trainset_y = traindata.iloc[:,1]
trainset_X = traindata.iloc[:, 11:]
testset_y = testdata.iloc[:,1]
testset_X = testdata.iloc[:,11:]
classifyset_X = PredictData.iloc[:,11:]
validset_X = ValidData.iloc[:,11:]


In [71]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(trainset_X,trainset_y,
                                                    test_size=0.3)
forest = RandomForestClassifier(criterion='entropy',
                                n_estimators=10, 
                                random_state=1,
                                n_jobs=2)

forestpredict = forest.fit(X_train, y_train)
forestpredict.score(X_test,y_test)
forestpredict.score(testset_X,testset_y)
# forestresult = forestpredict.predict(X_test)
# print(forestresult)


0.8368180390240497

In [72]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

X_train, X_test, y_train, y_test = train_test_split(trainset_X,trainset_y,
                                                    test_size=0.3)
clf = GaussianNB()

GNBclassify = clf.fit(X_train, y_train)
GNBclassify.score(X_test,y_test)
GNBclassify.score(testset_X,testset_y)
# GNBresult = GNBclassify.predict(X_test)
# print(GNBresult)
# (貝氏比較準)


0.9657928723515655

In [73]:
# Use Model to classify Unclassified Data
PredictData["category"] = GNBclassify.predict(classifyset_X)

In [74]:
# Use Model to classify Validation data
ValidData["category"] = GNBclassify.predict(validset_X)

To save in Mongo DB

In [ ]:
def saving_data(Data):
    politics_data_tmp = PredictData[(Data['category']=="politics")]
    politics_data_df = pd.DataFrame(politics_data_tmp,columns = ["_id","category","content","datetime","hash","id","location",
                  "media","reporter","title","url"])
    politics_data_dict = politics_data_df.to_dict('records')
    client = MongoClient('mongodb://192.168.87.135:27017/')
    db = client.news
    for x in politics_data_dict:
        posts = db.ValidationPure
        post_id = posts.insert_one(x).inserted_id
        
saving_data(PredictData)
saving_data(ValidData)
        

In [ ]:
import _pickle
with open("bayes_model","wb") as fp:
    pickle.dump(GNBclassify,fp)